In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Load the data

In [3]:
# Load the data

# Get current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "data/mnist.npz")

# Get only training set
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path)

11501568/11490434 [==============================] - 1s 0us/step


## Pre-processing the data
One important step when dealing with image data is to preprocess the data. During the preprocess step you can apply transformations to the dataset that will be fed into your convolutional neural network.

Here you will apply two transformations to the data:

Reshape the data so that it has an extra dimension. The reason for this is that commonly you will use 3-dimensional arrays (without counting the batch dimension) to represent image data. The third dimension represents the color using RGB values. This data might be in black and white format so the third dimension doesn't really add any additional information for the classification process but it is a good practice regardless.
Normalize the pixel values so that these are values between 0 and 1. You can achieve this by dividing every value in the array by the maximum.
Remember that these tensors are of type numpy.ndarray so you can use functions like reshape or divide to complete the reshape_and_normalize function below:

In [4]:
# GRADED FUNCTION: reshape_and_normalize

def reshape_and_normalize(images):

    ### START CODE HERE

    # Reshape the images to add an extra dimension
    images = tf.expand_dims(images, axis = -1)

    # Normalize pixel values
    images = images / 255

    ### END CODE HERE

    return images

In [5]:
# Reload the images in case you run this cell multiple times
(training_images, _), _ = tf.keras.datasets.mnist.load_data(path=data_path)

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


## Defining your callback
Now complete the callback that will ensure that training will stop after an accuracy of 99.5% is reached.

Define your callback in such a way that it checks for the metric accuracy (acc can normally be used as well but the grader expects this metric to be called accuracy so to avoid getting grading errors define it using the full word).

In [11]:
# GRADED CLASS: myCallback
### START CODE HERE
from keras.callbacks import Callback
# Remember to inherit from the correct class
class myCallback(Callback):
    # Define the method that checks the accuracy at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 99.5:
            self.model.stop_training = True

### END CODE HERE

In [12]:
# GRADED FUNCTION: convolutional_model
def convolutional_model():
    ### START CODE HERE
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
    # Define the model
    model = tf.keras.models.Sequential([
    Conv2D(64, (3, 3), activation = "relu", input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation = "relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation = "relu"),
    Dense(10, activation = "softmax")
    ])

    ### END CODE HERE

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [13]:
# Save your untrained model
model = convolutional_model()

# Get number of weights
model_params = model.count_params()

# Unit test to limit the size of the model
assert model_params < 1000000, (
    f'Your model has {model_params:,} params. For successful grading, please keep it '
    f'under 1,000,000 by reducing the number of units in your Conv2D and/or Dense layers.'
)

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.1358 - accuracy: 0.9581
Epoch 2/10
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0453 - accuracy: 0.9862
Epoch 3/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0312 - accuracy: 0.9901
Epoch 4/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0236 - accuracy: 0.9923
Epoch 5/10
1875/1875 [==============================] - 29s 15ms/step - loss: 0.0167 - accuracy: 0.9948
Epoch 6/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0134 - accuracy: 0.9958
Epoch 7/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0107 - accuracy: 0.9964
Epoch 8/10
1875/1875 [==============================] - 29s 15ms/step - loss: 0.0079 - accuracy: 0.9975
Epoch 9/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0082 - accuracy: 0.9975
Epoch 10/10
1875/1875 [==============================] - 31s 16m

In [9]:
print(f"Your model was trained for {len(history.epoch)} epochs")

Your model was trained for 10 epochs


In [10]:
if not "accuracy" in history.model.metrics_names:
    print("Use 'accuracy' as metric when compiling your model.")
else:
    print("The metric was correctly defined.")

The metric was correctly defined.
